In [2]:
from __init__ import *
from natsort import natsorted


import snmcseq_utils
import subprocess
import CEMBA_organize_files

In [3]:

log = snmcseq_utils.create_logger()
log.info('.')

04/09/2018 09:18:12 PM .


In [5]:

dataset = 'MB_v1'
orig_allcdir = '/cndd/fangming/snmcseq_dev/data/allc/{}'.format(dataset)
path_dst = '/cndd/Public_Datasets/human_snmcseq/Datasets/{}'.format(dataset)

with snmcseq_utils.cd(orig_allcdir):
    allc_dirs = sorted(glob.glob("*")) 
    for i, allc_dir in enumerate(allc_dirs):
        logging.info("Progress: {}/{}".format(i+1, len(allc_dirs)))
        # allc_dir is cell name
        dst = os.path.join(path_dst, 'allc', 'allc_'+allc_dir+'.tsv')
        with snmcseq_utils.cd(allc_dir):
            allc_files = natsorted(glob.glob("allc_*.tsv.gz"))
            assert len(allc_files) == 24

            for allc_file in allc_files:
                # decompress, remove first line, and  append
                cmd = 'gzip -cd {} | tail -n +2 >> {}'.format(allc_file, dst)
                os.system(cmd)

            # compress
            cmd = 'bgzip -c {} > {}'.format(dst, dst+'.bgz') 
            os.system(cmd)
           
            # remove
            cmd = 'rm {}'.format(dst)
            os.system(cmd)

            # tabix
            CEMBA_organize_files.tabix_index_allc(dst+'.bgz', skip_lines=0)

 

04/09/2018 09:21:56 PM Progress: 1/2784
04/09/2018 09:22:40 PM Progress: 2/2784


In [23]:

f = '/cndd/Public_Datasets/human_snmcseq/Ensembles/Ens0/cluster/clusters_v1_binc_mCH_louvain.tsv'
df = pd.read_table(f)
f = '/cndd/Public_Datasets/human_snmcseq/Ensembles/Ens0/cluster/clusters_v1_binc_mCH_louvain_annotation.tsv'
df_annot = pd.read_table(f)
df = pd.merge(df, df_annot, on='cluster_ID')

df['dataset'] = df['sample'].apply(lambda x: '_'.join(x.split('_')[1:3]))
df['cluster_ID'] = df['cluster_ID'].apply(lambda x: int(x.strip('cluster_')))
df = df.rename(columns={'sample': 'cell_name', 
                        'cluster_ID': 'cluster', 
                        'cluster_annotation': 'annotation'})
df = df[['cell_name', 'dataset', 'cluster', 'annotation']].fillna('Glia/Unlabeled')

print(df.shape)
df.head()

(6435, 4)


,cell_name,dataset,cluster,annotation
0,160729_MB_v1_hs_25yr_MFG_pool_1000_AD008_indexed,MB_v1,8,hSst-2
1,160729_MB_v1_hs_25yr_MFG_pool_1019_AD006_indexed,MB_v1,8,hSst-2
2,160729_MB_v1_hs_25yr_MFG_pool_101_AD010_indexed,MB_v1,8,hSst-2
3,160729_MB_v1_hs_25yr_MFG_pool_1022_AD006_indexed,MB_v1,8,hSst-2
4,160729_MB_v1_hs_25yr_MFG_pool_1033_AD002_indexed,MB_v1,8,hSst-2


In [24]:
df.to_csv('/cndd/Public_Datasets/human_snmcseq/Ensembles/Ens0/Ens0.tsv',
         sep='\t', na_rep='NA', header=True, index=False)